# 🤖 4º SPRINT - IA


INTEGRANTES:

Julia Monteiro RM: 557023

Sofia Petruk RM: 556585

Victor Henrique Estrella Carracci RM: 556206


▶ *EXPLICAÇÃO DOS DADOS E DISCUSSÃO*

O processo de desenvolvimento do modelo de classificação para o aplicativo "Descomplica Auto" envolveu várias etapas cruciais, desde a seleção e preparação dos dados até a validação do modelo. Utilizando um conjunto de dados com 189 registros, que incluíam informações sobre a peça do veículo, sinais de falha, resultados esperados, quilometragem e condições de uso, conseguimos treinar um modelo baseado em Random Forest.

Ao analisar a importância das características, observamos que a quilometragem (32,88%) e o sinal (31,97%) foram as variáveis mais relevantes na determinação do resultado, seguidas pelo nome da peça (23,24%) e pelas condições de uso (11,91%). Essa distribuição reflete a lógica prática de que a condição de uso e os sinais percebidos são fundamentais na avaliação do estado do veículo.


Os resultados do primeiro código foram muito promissores com um total de 100% de acuracio, portanto a IA não estava agindo corretamente na hora de testar a api, ele retornava um resultado diferente pois ele estava levando muiuto em consideração a feature de quilometragem, consequentemente foi feito alterações no códigos para conseguir alterar essa parte. Para isso restingimos a profundidade da árvore, para que não ocorresse o overfitting. Resultando em um novo modelo com 97% de acurácia e a matriz de confusão revelou que a maioria das classes foi classificada corretamente, refletindo uma precisão e recall elevados, principalmente nas classes com maior suporte.



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.preprocessing import LabelEncoder

TESTE PARA VIZUALIZAR IMPORTÂNCIA DAS FEATURES

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('/content/dataset.csv')

label_encoders = {}
for column in ['Nome da Peça', 'Sinal', 'Condições de Uso']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

X = data[['Nome da Peça', 'Sinal', 'Quilometragem', 'Condições de Uso']]
y = data['Resultado']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

importances = model.feature_importances_
feature_importances = pd.DataFrame(importances, index=X.columns, columns=['importance']).sort_values('importance', ascending=False)

print(feature_importances)


  Nome da Peça                   Sinal                          Resultado  \
0        Motor                  fumaça          Superaquecimento do motor   
1        Motor                 barulho   Desgaste nos rolamentos do motor   
2        Motor  dificuldade de partida  Problemas na bomba de combustível   
3        Motor                vibração          Desbalanceamento do motor   
4        Motor  ruído de funcionamento        Desgaste das peças internas   

   Quilometragem Condições de Uso  
0         150000       uso urbano  
1         120000        uso misto  
2          90000       uso urbano  
3         130000   uso em estrada  
4         110000       uso urbano  
Dimensão de X: (189, 4), Dimensão de y: (189,)
                  importance
Quilometragem       0.328843
Sinal               0.319723
Nome da Peça        0.232365
Condições de Uso    0.119069


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import RandomOverSampler

df = pd.read_csv('/content/dataset.csv')

X = df.drop(columns=['Resultado'])
y = df['Resultado']

X_encoded = pd.get_dummies(X, drop_first=True)

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_encoded, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

rf = RandomForestClassifier(random_state=42, class_weight='balanced')

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

class_counts = y_resampled.value_counts()
min_class_size = class_counts.min()
n_splits = min(5, min_class_size)

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=skf, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_

y_pred = best_rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Acurácia do modelo: {accuracy:.4f}")
print("Matriz de Confusão:")
print(conf_matrix)
print("Relatório de Classificação:")
print(class_report)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Nome da Peça      189 non-null    object
 1   Sinal             189 non-null    object
 2   Resultado         189 non-null    object
 3   Quilometragem     189 non-null    int64 
 4   Condições de Uso  189 non-null    object
dtypes: int64(1), object(4)
memory usage: 7.5+ KB
None
Distribuição das classes após reamostragem:
Resultado
Superaquecimento do motor               7
Filtro de ar obstruído                  7
Eixo com folga nas juntas               7
Desgaste nos suportes do eixo           7
Eixo comprometido                       7
                                       ..
Vazamento na bomba de combustível       7
Bomba com ar na linha de combustível    7
Problemas na válvula de combustível     7
Bomba desalinhada                       7
Roda com desgaste irregular             7
Nam

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
540 fits failed out of a total of 1080.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
74 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-pa

Acurácia do modelo: 1.0000
Matriz de Confusão:
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 2]]
Relatório de Classificação:
                                                precision    recall  f1-score   support

                 Amortecedor com baixa pressão       1.00      1.00      1.00         1
                      Amortecedor com desgaste       1.00      1.00      1.00         1
                      Amortecedor comprometido       1.00      1.00      1.00         2
                        Amortecedor danificado       1.00      1.00      1.00         2
                        Ar no sistema de freio       1.00      1.00      1.00         3
                Bateria com carga insuficiente       1.00      1.00      1.00         1
                     Bateria com falha interna       1.00      1.00      1.00         1
                         Bateria com problemas       1.00      1.00      1.00         1
            

In [ ]:
import pickle

with open('random_forest_model.pkl', 'wb') as model_file:
    pickle.dump(best_rf, model_file)


In [ ]:
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)


In [ ]:

with open('colunas_treinamento.pkl', 'wb') as colunas_file:
    pickle.dump(X_encoded.columns, colunas_file)


# **RESTRIÇÃO DE PROFUNDIDADE/PARAMETROS**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import RandomOverSampler

df = pd.read_csv('/content/dataset.csv')
df = df.drop(columns=['Quilometragem'])

X = df.drop(columns=['Resultado'])
y = df['Resultado']

X_encoded = pd.get_dummies(X, drop_first=True)

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_encoded, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

rf = RandomForestClassifier(random_state=42, class_weight='balanced')

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [2, 4],
    'max_features': ['sqrt']
}

class_counts = y_resampled.value_counts()
min_class_size = class_counts.min()
n_splits = min(5, min_class_size)

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=skf, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_

y_pred = best_rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Acurácia do modelo: {accuracy:.4f}")
print("Matriz de Confusão:")
print(conf_matrix)
print("Relatório de Classificação:")
print(class_report)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Nome da Peça      189 non-null    object
 1   Sinal             189 non-null    object
 2   Resultado         189 non-null    object
 3   Quilometragem     189 non-null    int64 
 4   Condições de Uso  189 non-null    object
dtypes: int64(1), object(4)
memory usage: 7.5+ KB
None
Distribuição das classes após reamostragem:
Resultado
Superaquecimento do motor               7
Filtro de ar obstruído                  7
Eixo com folga nas juntas               7
Desgaste nos suportes do eixo           7
Eixo comprometido                       7
                                       ..
Vazamento na bomba de combustível       7
Bomba com ar na linha de combustível    7
Problemas na válvula de combustível     7
Bomba desalinhada                       7
Roda com desgaste irregular             7
Nam

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Acurácia do modelo: 0.9745
Matriz de Confusão:
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 2]]
Relatório de Classificação:
                                                precision    recall  f1-score   support

                 Amortecedor com baixa pressão       1.00      1.00      1.00         1
                      Amortecedor com desgaste       1.00      1.00      1.00         1
                      Amortecedor comprometido       1.00      1.00      1.00         2
                        Amortecedor danificado       1.00      1.00      1.00         2
                        Ar no sistema de freio       1.00      1.00      1.00         3
                Bateria com carga insuficiente       1.00      1.00      1.00         1
                     Bateria com falha interna       1.00      1.00      1.00         1
                         Bateria com problemas       1.00      1.00      1.00         1
            

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

In [ ]:
import pickle


with open('random_forest_model.pkl', 'wb') as model_file:
    pickle.dump(best_rf, model_file)


with open('colunas_treinamento.pkl', 'wb') as colunas_file:
    pickle.dump(X_encoded.columns, colunas_file)